# Hotel Booking System

### Functional Requirement

Hotel Booking System

Functional Requirement

1. Welcome the user (Hello name, Welcome to HotelReserve.com.)
2. Location - User sees all available locations where we have presence
or User inputs a location 
3. Account Registration - User inputs First name,Last name,Mobile number, 
email address
4. Hotel Booking - user see all hotels with address available in the selected location with price range from minimum to maximum,
user selects a hotel and type of room(studio,deluxe), user inputs a check in and check out date. If room is available for those days, user proceeds to enter number of guests and the system displays the total price(rate per room * number of lodging days). User confirms booking or exits if not interested.


In [ ]:
#importing libraries

import random
import json
import requests
from datetime import datetime
import pandas as pd
from IPython.display import display

In [ ]:
def get_destination_id(city):
    '''
    This function accepts a parameter(city name) 
    and returns city names with associated 
    destination id.
    '''
    
    hotel_log = {}
    
    url = "https://hotels-com-provider.p.rapidapi.com/v1/destinations/search"

    querystring = {"currency":"USD","query":{city},"locale":"en_US"}

    headers = {
        'x-rapidapi-key': "80495dedf0msh1c0851b749bbd94p1e2f97jsnbe8eee2309d9",
        'x-rapidapi-host': "hotels-com-provider.p.rapidapi.com"
        }

    response = requests.request("GET", url, headers=headers, params=querystring)



    dic = json.loads(response.text)
    city_list = dic['suggestions'][0]['entities']
     
    for i in range(len(city_list)):
        id_log = []
        for name,element in city_list[i].items():
            if name == 'destinationId':
                id_log.append(element)
            if name == 'name':
                id_log.append(element)
                hotel_log[i] = id_log
    city_dtf = pd.DataFrame(hotel_log).transpose()
    city_dtf.rename(columns={0:"destination_id", 1:"City"}, inplace=True)
    return city_dtf

In [ ]:
def hotels_list(destination_id,checkOut_date,checkIn_date):
    '''
    This function returns hotel id,name and address
    given destination id, check-in and check-out dates.
    '''
    
    hotel_log = {}
    
    url = "https://hotels4.p.rapidapi.com/properties/list"

    querystring = {"adults1":"1","pageNumber":"1","destinationId":{destination_id},"pageSize":"25","checkOut":{checkOut_date},"checkIn":{checkIn_date},"sortOrder":"PRICE","locale":"en_US","currency":"USD"}

    headers = {
        'x-rapidapi-key': "80495dedf0msh1c0851b749bbd94p1e2f97jsnbe8eee2309d9",
        'x-rapidapi-host': "hotels4.p.rapidapi.com"
        }

    response = requests.request("GET", url, headers=headers, params=querystring)

    dic = json.loads(response.text)
    hotel_list = dic["data"]["body"]["searchResults"]["results"]

    for i in range(len(hotel_list)):
        hotel_details_list = []
        for name,element in hotel_list[i].items():
            if name == 'id':
                hotel_details_list.append(element)
            elif name == 'name':
                hotel_details_list.append(element)
            elif name == 'starRating':
                hotel_details_list.append(element)
            elif name == 'address':
                delem = str(element.get("streetAddress")) + "," + str(element.get("region"))
                hotel_details_list.append(delem)
                hotel_log[i] = hotel_details_list
    hotel_dtf = pd.DataFrame(hotel_log).transpose()
    hotel_dtf.rename(columns={0:"hotel_id", 1:"Hotel_Names", 2:"Rating", 3:"Address"}, inplace=True)
    return hotel_dtf

In [ ]:
def get_hotel_details(id):
    
    url = "https://hotels4.p.rapidapi.com/properties/get-details"

    querystring = {"id":id}

    headers = {
    'x-rapidapi-key': "dd73f97d50msh50271603b0c3fd0p1e6579jsn65e1574aabdd",
    'x-rapidapi-host': "hotels4.p.rapidapi.com"
    }

    response_details = requests.request("GET", url, headers=headers, params=querystring)
    
    #hotel overview
    overview = response_details.json()['data']['body']['overview']['overviewSections']
    for view in overview:
        if 'title' in view:
            print('\033[1m' + view['title'] + '\033[0m')
            print(*view['content'], '\n', sep = '\n')

    #hygiene
    try:
        hygieneAndCleanliness = response_details.json()['data']['body']['hygieneAndCleanliness']
        print('\033[1m' + hygieneAndCleanliness['title'] + '\033[0m')
        print(*hygieneAndCleanliness['healthAndSafetyMeasures']['measures'], '\n', sep = '\n')
    except:
        print("Unavailable")
    
    #room type
    roomTypes = response_details.json()['data']['body']['propertyDescription']['roomTypeNames']
    print('\033[1m' + 'Room Types' + '\033[0m')
    print(*roomTypes, sep = '\n')
    
    #Average Price per night
    price = response_details.json()['data']['body']['propertyDescription']['featuredPrice']['currentPrice']['plain']
    print('\n\033[1m' + 'Current Average Price' + '\033[0m')
    print(f"$ {price}")

In [ ]:
def booking_reg():
    
    #Other Personal Details
    mobile_num = input("Mobile Number (+234 8137189101): ")
    email = input("Email_Address: ")
    
    #Date
    checkin_date = str(input("Enter your checkin date (YYYY-MM-DD): "))
    checkout_date = str(input("Enter your checkout date (YYYY-MM-DD): "))
    date_format = "%Y-%m-%d"
    start = datetime.strptime(checkin_date, date_format)
    end = datetime.strptime(checkout_date, date_format)
    total_days = (end - start).days
    
    #Price
    url = "https://hotels4.p.rapidapi.com/properties/get-details"

    querystring = {"id":id}

    headers = {
    'x-rapidapi-key': "dd73f97d50msh50271603b0c3fd0p1e6579jsn65e1574aabdd",
    'x-rapidapi-host': "hotels4.p.rapidapi.com"
    }

    response_details = requests.request("GET", url, headers=headers, params=querystring)
    
    price = response_details.json()['data']['body']['propertyDescription']['featuredPrice']['currentPrice']['plain']
    total_price = price * total_days
    
    print(f'Hello {first_name}!!\nBooking cost for {total_days}days is {total_price}')

In [ ]:
hotel_reserve_log = {}

def booking_confirmation(hotel_id,first_name):
    num = random.randint(100,600)
    booking_id = first_name[1:4] + str(num)
    print(f'Hello{first_name}!!\nHotel ID {hotel_id} successful booked.\nYour booking ID is {booking_id}')
    hotel_reserve_log[booking_id] = {"Name": first_name, "ID":booking_id}
    return

In [ ]:
def hotel_log(log):
    
    hotel_log_dtf = pd.DataFrame(log)
    
    return hotel_log_dtf

In [ ]:
def hotel_booking_system():
    '''
    This function is a travel reservation system that ...
    '''
    
    print("Welcome to easYtravel.com")
    runtime = 1
    while runtime:
        checker1 = input("Enter 0 for user login, 1 for Employee Login, 2 to Quit : ")
        if checker1 == "0":
            checker2 = input("Enter 0 for hotel booking, 1 to Quit : ")
            if checker2 == "0":
                des_city = str(input("Enter your desired destination (city): "))
                display(get_destination_id(des_city))
                dest_id = str(input("Enter your destination id number from the above list: "))
                checko_date = str(input("Enter your checkout date (YYYY-MM-DD): "))
                checki_date = str(input("Enter your checkin date (YYYY-MM-DD): "))
                display(hotels_list(dest_id,checko_date,checki_date))
                hotel_id = str(input("Enter your prefered hotel id number: "))
                print('\033[1m' + 'Hotel Overview' + '\033[0m')
                get_hotel_details(hotel_id)
                  
                checker3 = input("Enter 0 to proceed with hotel booking, 1 to Quit : ")
                if checker3 == "0":
                      first_name = input("First name: ")
                      last_name = input("Last name: ")
                      booking_reg()
                      checker4 = input("Enter 0 to confirm hotel booking, 1 to Quit : ")
                      if checker4 == "0":
                          booking_confirmation(hotel_id,first_name)
                      else:
                          break 
                else:
                      break
                  
        elif checker1 == "1":
            print('\033[1m' + "Hotel Log" + '\033[0m')
            display(hotel_log(hotel_reserve_log))
        elif checker1 == "2":
            break    
        else:
            print("Invalid value, try again")
            hotel_booking_system()

In [ ]:
hotel_booking_system()